In [1]:
import datetime
from xml.etree import ElementTree as ET
import requests
from requests.auth import HTTPDigestAuth
import numpy as np
import time
import json

In [2]:
# Most likely will need to add more key variables. Need to move two CT's from the 480 circuit to the 240V and then we can work only with one e-gauge

# If power is negative it means it is consuming power from grid
class EgaugeInterface():
    
    def __init__(self, url = None, username = None, password = None, t_sample = 5):
        # Initializing credentials
        self.url = url
        self.username = username
        self.password = password
        
        # Initializing parameters
        self.t_sample = t_sample
        self.keys = ['L1 Voltage', 'L2 Voltage', 'Power Circuit 1', 'Power Circuit 1*', 'Power Circuit 2',
                'Power Circuit 2*', 'Power Circuit 1 neutral', 'Shed Power', 'Control Fan Power', 'Control Fan Power*', 'ts']
        
        
    # Function to get and format e-gauge data
    def get_egauge_data(self, request):
        power_values = dict.fromkeys(self.keys, None)

        root = ET.fromstring(request.text)
        timestamp  = root.findtext("ts")

        for r in root.findall('r'):
            for child in r:
                if r.get('n') == 'L1 Voltage':
                    power_values['L1 Voltage'] = (int(child.text))
                elif r.get('n') == 'L2 Voltage':
                    power_values['L2 Voltage'] = (int(child.text))
                elif r.get('n') == 'Power Circuit 1':
                    power_values['Power Circuit 1'] = (int(child.text))
                elif r.get('n') == 'Power Circuit 1*':
                    power_values['Power Circuit 1*'] = (int(child.text))
                elif r.get('n') == 'Power Circuit 2':
                    power_values['Power Circuit 2'] = (int(child.text))
                elif r.get('n') == 'Power Circuit 2*':
                    power_values['Power Circuit 2*'] = (int(child.text))
                elif r.get('n') == 'Power Circuit 1 neutral':
                    power_values['Power Circuit 1 neutral'] = (int(child.text))
                elif r.get('n') == 'Shed Power':
                    power_values['Shed Power'] = (int(child.text))
                elif r.get('n') == 'Control Fan Power':
                    power_values['Control Fan Power'] = (int(child.text))
                elif r.get('n') == 'Control Fan Power*':
                    power_values['Control Fan Power*'] = (int(child.text))

        power_values['ts'] = int(root.findtext("ts"))
        return power_values

    
    # Function to process data from e-gauge and convert to useful power values
    def processing_egauge_data(self):
        power_values = dict.fromkeys(self.keys, None)
        try:
            resp = requests.get(self.url, auth=HTTPDigestAuth(self.username, self.password))
            data_ini = self.get_egauge_data(resp)

        except Exception as exc:
            print(exc)
            return json.dumps(power_values)

        time.sleep(self.t_sample)

        try:
            resp = requests.get(self.url, auth=HTTPDigestAuth(self.username, self.password))
            data_end = self.get_egauge_data(resp)

        except Exception as exc:
            print(exc)
            return json.dumps(power_values)

        power_values['ts'] = datetime.datetime.fromtimestamp(int(data_end['ts'])).strftime('%Y-%m-%d %H:%M:%S')
        ts_delta = data_end['ts'] - data_ini['ts']
        power_values['L1 Voltage'] = ((data_end['L1 Voltage'] - data_ini['L1 Voltage'])/ts_delta)/1000
        power_values['L2 Voltage'] = ((data_end['L2 Voltage'] - data_ini['L2 Voltage'])/ts_delta)/1000
        power_values['Power Circuit 1'] = (data_end['Power Circuit 1'] - data_ini['Power Circuit 1'])/ts_delta
        power_values['Power Circuit 1*'] = (data_end['Power Circuit 1*'] - data_ini['Power Circuit 1*'])/ts_delta
        power_values['Power Circuit 2'] = (data_end['Power Circuit 2'] - data_ini['Power Circuit 2'])/ts_delta
        power_values['Power Circuit 2*'] = (data_end['Power Circuit 2*'] - data_ini['Power Circuit 2*'])/ts_delta
        power_values['Power Circuit 1 neutral'] = (data_end['Power Circuit 1 neutral'] - data_ini['Power Circuit 1 neutral'])/ts_delta
        power_values['Shed Power'] = (data_end['Shed Power'] - data_ini['Shed Power'])/ts_delta
        power_values['Control Fan Power'] = (data_end['Control Fan Power'] - data_ini['Control Fan Power'])/ts_delta
        power_values['Control Fan Power*'] = (data_end['Control Fan Power*'] - data_ini['Control Fan Power*'])/ts_delta

        json_dict = json.dumps(power_values)
        print('Data Dict: ', json_dict)
    #     return json_dict
        return power_values

In [13]:
# Testing
# define user, password and url

# user = 
# password = 
# url = 

power = EgaugeInterface(url=url, username=user, password=password).processing_egauge_data()

Data Dict:  {"L1 Voltage": 238.03733333333335, "L2 Voltage": 239.03333333333333, "Power Circuit 1": 511.6666666666667, "Power Circuit 1*": 1105.0, "Power Circuit 2": 374.8333333333333, "Power Circuit 2*": 920.6666666666666, "Power Circuit 1 neutral": 289.5, "Shed Power": 680.3333333333334, "Control Fan Power": 19400.166666666668, "Control Fan Power*": 28611.5, "ts": "2019-05-23 14:16:09"}


In [14]:
power

{'L1 Voltage': 238.03733333333335,
 'L2 Voltage': 239.03333333333333,
 'Power Circuit 1': 511.6666666666667,
 'Power Circuit 1*': 1105.0,
 'Power Circuit 2': 374.8333333333333,
 'Power Circuit 2*': 920.6666666666666,
 'Power Circuit 1 neutral': 289.5,
 'Shed Power': 680.3333333333334,
 'Control Fan Power': 19400.166666666668,
 'Control Fan Power*': 28611.5,
 'ts': '2019-05-23 14:16:09'}

In [8]:
power

{'L1 Voltage': 237.434,
 'L2 Voltage': 238.28816666666665,
 'Power Circuit 1': 208.5,
 'Power Circuit 1*': 603.3333333333334,
 'Power Circuit 2': 135.33333333333334,
 'Power Circuit 2*': 426.8333333333333,
 'Power Circuit 1 neutral': 77.33333333333333,
 'Shed Power': 599.1666666666666,
 'Control Fan Power': 19304.166666666668,
 'Control Fan Power*': 28570.0,
 'ts': '2019-05-23 14:07:15'}